In [1]:
from skt.gcp import (
    PROJECT_ID,
    bq_insert_overwrite,
    bq_to_df,
    bq_to_pandas,
    get_bigquery_client,
    bq_table_exists,
    get_max_part,
    load_query_result_to_table,
    pandas_to_bq,
    pandas_to_bq_table,
    load_bigquery_ipython_magic,
    get_bigquery_client,
    _print_query_job_results,
    load_query_result_to_partitions
    
)

from skt.ye import (
    get_hdfs_conn,
    get_spark,
    hive_execute,
    hive_to_pandas,
    pandas_to_parquet,
    slack_send,
    get_secrets
)

In [ ]:
from google.cloud.bigquery.job import QueryJobConfig

In [2]:
from datetime import datetime, timedelta

In [92]:
execution_dt = datetime.strptime(current_dt, '%Y-%m-%d')
execution_dt_next = (execution_dt + timedelta(days=1))
current_dt_next = execution_dt_next.strftime('%Y-%m-%d')
ttl = int(ttl)
print(f'execution_dt: {current_dt}')
print(f"state: {state}")
print(f"ttl: {ttl}")

execution_dt: 2024-07-08
state: stg
ttl: 30


In [106]:
""" no partition data in comm (snapshot) """
PROJECT_ID = "skt-datahub"
db_name  = "comm"
tmbr_meta_tbl= "mp_taxonomies_brand" 
tmbr_meta_map_tbl = "mp_taxonomies_brandMapp"

In [126]:
temp_db = 'temp_1d'

In [107]:
save_db_name = 'adot_reco_dev'
save_meta_table = 'tmbr_meta_mapping_tbl'

In [108]:
bq_clinet = get_bigquery_client()

### UPDATED META 임시테이블에 저장


In [ ]:
recent_meta_dt = get_max_part(f"{PROJECT_ID}.{save_db_name}.{save_meta_table}")

In [ ]:
query = f"""
    SELECT  *
    FROM    {db_name}.{tmbr_meta_tbl}
    WHERE brand_name is not null and brand_name !='' and LOWER(brand_name) NOT LIKE '%test%'
    AND updated_at >= '{recent_meta_dt}' or created_at >= '{recent_meta_dt}'
"""
job_config = QueryJobConfig()
temp_update_meta_table = f'{PROJECT_ID}.{temp_db}.tmbr_update_meta'
job_config.destination = temp_update_meta_table
job_config.write_disposition = 'WRITE_TRUNCATE'
# 임시 테이블 생성
query_job = bq_clinet.query(query, job_config=job_config)
query_job.result() 

In [159]:
## Meta table 전처리 로직: 중복 product 처리 (제거 x -> 메타 합산)
query = f"""

WITH META_TABLE AS (
    SELECT  brand_id,
            count(distinct brand_name) over(partition by COALESCE(TRIM(LOWER(brand_name)), '')) as brand_cnt,
            COALESCE(TRIM(LOWER(brand_name)), '') as brand_name,
            del_yn,
            display_name
            prm_brand_name,
            categories,
            COALESCE(TRIM(LOWER(category_large_name)), '') as category_large_name,
            COALESCE(TRIM(LOWER(category_medium_name)), '') as category_medium_name,
            COALESCE(TRIM(LOWER(category_small_name)), '') as category_small_name,
            COALESCE(TRIM(LOWER(description)), '') as description

    FROM    {db_name}.{tmbr_meta_tbl}
    WHERE   dt = '{recent_meta_dt}'
            AND brand_name is not null and brand_name !='' and LOWER(brand_name) NOT LIKE '%test%'
),

DUPLICATED_BRAND_TABLE AS(
    SELECT  brand_name, 
            ARRAY_AGG(category_large_name) as category_large_names,
            ARRAY_AGG(category_medium_name) as category_medium_names,
            ARRAY_AGG(category_small_name) as category_small_names,
            ARRAY_AGG(description) as descriptions

    FROM (
        SELECT  brand_name,
                category_large_name,
                category_medium_name,
                category_small_name, 
                description

        FROM META_TABLE
        WHERE brand_name !='' AND brand_cnt > 1 
    )
    GROUP BY brand_name
),

NON_DUPLICATED_BRAND_TABLE AS(
    SELECT  brand_name,
            category_large_name,
            category_medium_name,
            category_small_name, 
            description
            

    FROM META_TABLE
    WHERE brand_name !='' AND brand_cnt = 1
),

MERGED_DUPLICATE_BRAND_TABLE AS(
    SELECT  brand_name,
            ARRAY_TO_STRING(category_large_names, ',') as category_large_name,
            ARRAY_TO_STRING(category_medium_names, ',') as category_medium_name,
            ARRAY_TO_STRING(category_small_names, ',') as category_small_name,
            ARRAY_TO_STRING(descriptions, ',') as description,
            
    FROM(
        SELECT  brand_name,
                ARRAY(
                    SELECT category_large_name
                    FROM UNNEST(category_large_names) AS category_large_name 
                    WHERE category_large_name != ''
                  ) AS category_large_names,
                ARRAY(
                    SELECT category_medium_name 
                    FROM UNNEST(category_medium_names) AS category_medium_name 
                    WHERE category_medium_name != ''
                  ) AS category_medium_names,

                ARRAY(
                    SELECT category_small_name
                    FROM UNNEST(category_small_names) AS category_small_name
                    WHERE category_small_name != ''
                  ) AS category_small_names,
                ARRAY(
                    SELECT description
                    FROM UNNEST(descriptions) AS description
                    WHERE description != ''
                  ) AS descriptions     

        FROM DUPLICATED_BRAND_TABLE
    )
)

SELECT  *
        
FROM MERGED_DUPLICATE_BRAND_TABLE

UNION ALL

SELECT  *

FROM NON_DUPLICATED_BRAND_TABLE
"""


In [160]:
from google.cloud.bigquery.job import QueryJobConfig

In [161]:
job_config = QueryJobConfig()
temp_no_duplicate_meta_table = f'{PROJECT_ID}.{temp_db}.tmbr_temp_table'
job_config.destination = temp_no_duplicate_meta_table
job_config.write_disposition = 'WRITE_TRUNCATE'

In [162]:
# 임시 테이블 생성
query_job = bq_clinet.query(query, job_config=job_config)
query_job.result() 

In [ ]:
query = f"""
WITH META_TABLE AS (
    SELECT  *
    FROM {temp_no_duplicate_meta_table}
),

SUB_WORD_TABLE AS (

    SELECT  brand_word,
            brand_name,
            category_large_name,
            category_medium_name,
            category_small_name,
            description,
            count(brand_word) over(partition by brand_word) as word_cnt
            
    FROM (
        SELECT  category_large_name,
                category_medium_name,
                category_small_name,
                description,
                brand_name,
                brand_word
                
        FROM META_TABLE,
        UNNEST(SPLIT(brand_name, ' ')) AS brand_word
    )
),

tmbr_etymology_table AS (
    SELECT  A.brand_name,
            A.category_large_name,
            A.category_medium_name,
            A.category_small_name,
            A.description,
            A.is_empty,
            B.del_yn
    FROM (

        SELECT distinct brand_name,
               category_large_name,
               category_medium_name,
               category_small_name,
               description,
               CASE 
                   WHEN category_medium_name !='' OR category_small_name !='' THEN 'N'
                   ELSE 'Y'
               END AS is_empty
               
        FROM SUB_WORD_TABLE
        WHERE brand_name = brand_word
    )AS A
    LEFT JOIN  (
        SELECT  distinct COALESCE(TRIM(LOWER(brand_name)), '') as brand_name,
                del_yn

        FROM  {db_name}.{tmbr_meta_tbl}
        WHERE brand_name is not null and brand_name !=''
    ) AS B
    ON A.brand_name = B.brand_name
),

derivative_table AS(
    SELECT  distinct    brand_word as brand_name,
                        brand_name as derivative_brand
    FROM SUB_WORD_TABLE
    WHERE brand_word in (SELECT distinct brand_name FROM tmbr_etymology_table) 
    AND brand_name != brand_word AND brand_word !=''
),

final_tmbr_etymology_table AS (

    SELECT distinct COALESCE(brand_name, '') as brand_name,
                    COALESCE(category_large_name, '') as category_large_name,
                    COALESCE(category_medium_name, '') as category_medium_name,
                    COALESCE(category_small_name, '') as category_small_name,
                    COALESCE(description, '') as description,
                    COALESCE(derivative_brand, '') as derivative_brand,
                    PARSE_DATE('%Y-%m-%d', '{current_dt_next}') as dt 
    FROM (
        SELECT  A.brand_name,
                A.category_large_name,
                A.category_medium_name,
                A.category_small_name,
                A.description,
                B.derivative_brand

        FROM tmbr_etymology_table AS A

        LEFT JOIN (
            SELECT  brand_name,
                    derivative_brand
            FROM derivative_table
        ) AS B
        ON A.brand_name = B.brand_name
    )
)
SELECT * 
FROM final_tmbr_etymology_table A
"""

In [ ]:
try:
    etymology_max_dt = get_max_part(f"{PROJECT_ID}.{save_db_name}.tmbr_etymology_table")
    query = query + f"""WHERE NOT EXISTS (
                SELECT 1
                FROM {PROJECT_ID}.{save_db_name}.tmbr_etymology_table AS B
                WHERE A.brand_name = B.brand_name AND A.category_medium_name = B.category_medium_name AND A.category_small_name = B.category_small_name
                      AND A.derivative_brand = B.derivative_brand 
                AND dt <= '{etymology_max_dt}'
            )"""
except ValueError as e:
    pass

In [ ]:
bq_insert_overwrite(sql=query, destination=f'{PROJECT_ID}.{save_db_name}.tmbr_etymology_table', partition='dt')

In [176]:
# query = f"""
# WITH META_TABLE AS (
#     SELECT  *
#     FROM {temp_no_duplicate_meta_table}
# ),

# SUB_WORD_TABLE AS (

#     SELECT  brand_word,
#             brand_name,
#             category_large_name,
#             category_medium_name,
#             category_small_name,
#             description,
#             count(brand_word) over(partition by brand_word) as word_cnt
            
#     FROM (
#         SELECT  category_large_name,
#                 category_medium_name,
#                 category_small_name,
#                 description,
#                 brand_name,
#                 brand_word
                
#         FROM META_TABLE,
#         UNNEST(SPLIT(brand_name, ' ')) AS brand_word
#     )
# ),

# tmbr_etymology_table AS (
#     SELECT  A.brand_name,
#             A.category_large_name,
#             A.category_medium_name,
#             A.category_small_name,
#             A.description,
#             A.is_empty,
#             B.del_yn
#     FROM (

#         SELECT distinct brand_name,
#                category_large_name,
#                category_medium_name,
#                category_small_name,
#                description,
#                CASE 
#                    WHEN category_medium_name !='' OR category_small_name !='' THEN 'N'
#                    ELSE 'Y'
#                END AS is_empty
               
#         FROM SUB_WORD_TABLE
#         WHERE brand_name = brand_word
#     )AS A
#     LEFT JOIN  (
#         SELECT  distinct COALESCE(TRIM(LOWER(brand_name)), '') as brand_name,
#                 del_yn

#         FROM  {db_name}.{tmbr_meta_tbl}
#         WHERE brand_name is not null and brand_name !=''
#     ) AS B
#     ON A.brand_name = B.brand_name
# ),

# derivative_table AS(
#     SELECT  brand_word,
#             ARRAY_TO_STRING(ARRAY_AGG(brand_name), '||') as derivative_brands
#     FROM(
#         SELECT  distinct brand_word,
#                          brand_name
#         FROM SUB_WORD_TABLE
#         WHERE brand_word in (SELECT distinct brand_name FROM tmbr_etymology_table) 
#         AND brand_name != brand_word AND brand_word !=''
#     )
#     GROUP BY brand_word
# )

# final_tmbr_etymology_table AS (

#     SELECT distinct *,
#            PARSE_DATE('%Y-%m-%d', '{current_dt_next}') as dt 
#     FROM (
#         SELECT  A.*,
#                 B.derivative_brands

#         FROM tmbr_etymology_table AS A

#         LEFT JOIN (
#             SELECT  brand_word,
#                     derivative_brands
#             FROM derivative_table
#         ) AS B
#         ON A.brand_name = B.brand_word
#     )
# )

# SELECT * 
# FROM final_tmbr_etymology_table A

# WHERE NOT EXISTS (
#     SELECT 1
#     FROM {PROJECT_ID}.{save_db_name}.tmbr_etymology_table
#     WHERE A.brand_name = B.brand_name
#     AND dt <= '{etymology_max_dt}'
# """

## Category mapping

In [ ]:
query = f"""
WITH META_TABLE AS (
    SELECT distinct *
    FROM {temp_no_duplicate_meta_table}
)

SELECT 
    brand_name,
    category_large_name,
    category_medium_name,
    category_small_name,
    description,
    PARSE_DATE('%Y-%m-%d', '{current_dt_next}') as dt 

FROM  META_TABLE
WHERE category_small_name !='' OR category_medium_name!=''
"""

In [ ]:
try:
    category_mapping_table_max_dt = get_max_part(f"{PROJECT_ID}.{save_db_name}.tmbr_category_mapping_table")
    query = query + f"""AND NOT EXISTS (
                SELECT 1
                FROM {PROJECT_ID}.{save_db_name}.tmbr_category_mapping_table AS B
                WHERE A.brand_name = B.brand_name AND A.category_medium_name = B.category_medium_name AND A.category_small_name = B.category_small_name
                AND dt <= '{etymology_max_dt}'
            )"""
except ValueError as e:
    pass

In [ ]:
bq_insert_overwrite(sql=query, destination=f'{PROJECT_ID}.{save_db_name}.tmbr_category_mapping_table', partition='dt')

In [179]:
query = f"""
WITH META_TABLE AS (
    SELECT distinct *
    FROM {temp_no_duplicate_meta_table}
),
TEMP_TABLE AS (
    SELECT  brand_name,
            category_large_name,
            CASE
                WHEN category_medium_name = '' AND category_small_name = '' THEN ''
                WHEN category_medium_name = '' THEN category_small_name
                WHEN category_small_name = '' THEN category_medium_name
                WHEN category_medium_name = category_small_name THEN category_medium_name
                WHEN STRPOS(category_medium_name, category_small_name) > 0 THEN category_small_name
                ELSE CONCAT(category_medium_name, ',', category_small_name)
            END AS categories,
            description
        
    FROM  META_TABLE
)

SELECT *
FROM TEMP_TABLE 
WHERE categories !=''
"""

In [180]:
categories_mapping_df = bq_to_pandas(query)

unsupported operand type(s) for /: 'NoneType' and 'int'
Downloading: 100%|██████████|


In [181]:
categories_mapping = categories_mapping_df.set_index('brand_name').to_dict()['categories']

In [182]:
import json
categories_mapping_json = json.dumps(categories_mapping, ensure_ascii=False)

In [199]:
query = f"""
CREATE TEMP FUNCTION get_categories(brand_name STRING)
RETURNS STRING
LANGUAGE js AS '''
  var root_product_mapping_json_set = {categories_mapping_json};
  return (brand_name in root_product_mapping_json_set) ? root_product_mapping_json_set[brand_name] : '';
''';

WITH META_TABLE AS (
    SELECT distinct *
    FROM {temp_no_duplicate_meta_table}
),

ARRAY_TABLE AS (
    SELECT *
    FROM (
        SELECT  *,
                CASE
                    WHEN category_medium_name = '' AND category_small_name = '' THEN ''
                    WHEN category_medium_name = '' THEN category_small_name
                    WHEN category_small_name = '' THEN category_medium_name
                    WHEN category_medium_name = category_small_name THEN category_medium_name
                    WHEN STRPOS(category_medium_name, category_small_name) > 0 THEN category_small_name
                    ELSE CONCAT(category_medium_name, ',', category_small_name)
                END AS categories,
                SPLIT(derivative_brands, '||') as derivative_brand_array
        FROM {PROJECT_ID}.{save_db_name}.tmbr_etymology_table
        WHERE dt <= '{current_dt_next}'
    )
),
exploded_data AS (
  SELECT 
    brand_name,
    description,
    is_empty,
    del_yn,
    categories,
    derivative_brand_element
  FROM ARRAY_TABLE,
  UNNEST(derivative_brand_array) AS derivative_brand_element
),

MERGED_TABLE AS (
    SELECT distinct brand_name,
                    CASE
                        WHEN categories = '' AND categories_from_derivative = '' THEN ''
                        WHEN categories = '' AND categories_from_derivative != '' THEN categories_from_derivative
                        ELSE categories
                    END AS categories,
                    description

    FROM (
        SELECT brand_name,
               description,
               categories,
               get_categories(derivative_brand_element) AS categories_from_derivative,
               del_yn
        FROM exploded_data
    )
    WHERE categories_from_derivative!='' OR categories!=''
)

SELECT *
FROM (
    SELECT  brand_name,
            CASE
                WHEN category_medium_name = '' AND category_small_name = '' THEN ''
                WHEN category_medium_name = '' THEN category_small_name
                WHEN category_small_name = '' THEN category_medium_name
                WHEN category_medium_name = category_small_name THEN category_medium_name
                WHEN STRPOS(category_medium_name, category_small_name) > 0 THEN category_small_name
                ELSE CONCAT(category_medium_name, ',', category_small_name)
            END AS categories,
            description

    FROM  META_TABLE
    WHERE brand_name not in (SELECT distinct brand_name FROM MERGED_TABLE)
)
WHERE categories!=''

UNION ALL
SELECT  *

FROM MERGED_TABLE
WHERE categories !=''
"""

In [200]:
job_config = QueryJobConfig()
temp_meta_preprocessed_table = f'{PROJECT_ID}.{temp_db}.tmbr_meta_preprocessed_table'
job_config.destination = temp_meta_preprocessed_table
job_config.write_disposition = 'WRITE_TRUNCATE'

In [201]:
query_job = bq_clinet.query(query, job_config=job_config)
query_job.result() 

# AGGREGATE

In [225]:
query = f"""
WITH category_split_table AS (
  SELECT 
    brand_name,
    SPLIT(categories, ',') as category_array
  FROM {PROJECT_ID}.{temp_db}.tmbr_meta_preprocessed_table
),

exploded_tables AS (
  SELECT 
    brand_name,
    category
  FROM category_split_table, UNNEST(category_array) AS category
),

categories_count_table AS (
  SELECT 
    brand_name,
    category,
    COUNT(*) as category_count
  FROM exploded_tables
  GROUP BY brand_name, category
),

ranked_items AS (
  SELECT 
    brand_name,
    category,
    category_count,
    ROW_NUMBER() OVER (PARTITION BY brand_name ORDER BY category_count DESC) as rank
  FROM categories_count_table
)

SELECT  B.brand_name,
        A.categories,
        REPLACE(TRIM(LOWER(A.brand_name)), ' ', '') as brand_name_model,
        B.brand_id,
        B.updated_at,
        B.created_at,
        PARSE_DATE('%Y-%m-%d', '{current_dt_next}') as dt 
FROM (
    SELECT brand_name,
           ARRAY_TO_STRING(ARRAY_AGG(DISTINCT category), ',') AS categories
    FROM ranked_items
    WHERE rank = 1 or category_count > 1
    GROUP BY brand_name 
) AS A

LEFT JOIN (
    SELECT *
    FROM  {db_name}.{tmbr_meta_tbl}
    WHERE   brand_status_type = 'ACTIVE'
            AND     del_yn = 'N'
) AS B
ON A.brand_name = TRIM(LOWER(B.brand_name))
"""

In [ ]:
bq_insert_overwrite(sql=query, destination=f'{PROJECT_ID}.{save_db_name}.tmbr_meta_mapping_tbl', partition='dt')

In [1]:
# job_config = QueryJobConfig()
# temp_meta_mapping_table = f'{PROJECT_ID}.{temp_db}.tmbr_meta_mapping_tbl'
# job_config.destination = temp_meta_mapping_table
# job_config.write_disposition = 'WRITE_TRUNCATE'

In [2]:
# query_job = bq_clinet.query(query, job_config=job_config)
# query_job.result() 

In [3]:
# meta_mapping_table_dt = get_max_part(f'{PROJECT_ID}.{save_db_name}.tmbr_meta_mapping_tbl')

In [ ]:
ttl_query = f"""
ALTER TABLE
  {PROJECT_ID}.{save_db_name}.tmbr_meta_mapping_tbl
SET
  OPTIONS(partition_expiration_days={ttl})
"""
bq_clinet.query(ttl_query).result()

# drop temp table code

In [ ]:
for temp_table_name in ["tmbr_etymology_table", "tmbr_meta_preprocessed_table"]:
    drop_query = f"""
    DROP TABLE IF EXISTS {PROJECT_ID}.{temp_db}.{temp_table_name}
    """
    bq_clinet.query(drop_query).result()